# Labeling ESA tables with VEC labels

In [3]:
import pandas as pd
import numpy as np
import os
from snorkel.labeling import labeling_function
from snorkel.labeling import LabelingFunction
from snorkel.labeling.lf.nlp import nlp_labeling_function
from snorkel.preprocess import preprocessor
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.apply.dask import DaskLFApplier
from sklearn.model_selection import train_test_split
import spacy
from spacy.matcher import Matcher
import numpy as np

from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizerFast, BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from transformers import AutoModelForSequenceClassification

import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy, f1, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, BackboneFinetuning, ModelPruning
RANDOM_SEED = 42
pl.seed_everything(RANDOM_SEED)

# weights and biases
import wandb

# lightning plus wandb
from pytorch_lightning.loggers import WandbLogger

Global seed set to 42


In [4]:
!nvidia-smi

Mon Jun  7 20:10:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.89       Driver Version: 465.89       CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA Quadro R... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   60C    P8     9W /  N/A |   1223MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
esa_df = pd.read_csv('./data/esa_index_with_table_text_no_labels.csv', index_col=False)

In [ ]:
esa_df.head()

In [22]:
esa_df.drop(columns=['Unnamed: 0', 'Index'], inplace=True)

In [23]:
esa_df.head()

,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,...,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,csvText
0,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-wor...,1059614_14_lattice-v_1.csv,Survey Date Aboriginal Communities De...
1,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_17_lattice-v_1.csv,Field Site No Name Legal Location ...
2,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,18,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_18_lattice-v_1.csv,Field Site No Name Legal Location ...
3,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,19,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_19_lattice-v_1.csv,Field Site No Name Legal Location ...
4,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,20,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_20_lattice-v_1.csv,Field Site No Name Legal Location ...


In [25]:
esa_df['csvText'].head()

0     Survey Date Aboriginal Communities Detail Jul...
1     Field Site No Name Legal Location UTM Coordin...
2     Field Site No Name Legal Location UTM Coordin...
3     Field Site No Name Legal Location UTM Coordin...
4     Field Site No Name Legal Location UTM Coordin...
Name: csvText, dtype: object

In [39]:
snorkel_df = esa_df[['Title', 'csvText', 'label']]

C:\Users\ThibJacq\Anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-39-b648c9c9aca6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snorkel_df['label'] = -1


We will now go into Label Studio and label 500 examples. We will use these labels to create weak labels with Snorkel. Essentially, we won't need to label the entire dataset manually if we use Snorkel. However, we will need to look at the least confident examples and potentially relabel any mistakes.

In [ ]:
labeled_df = pd.read_csv('./mydata/esa_index_labeled.csv')

Here we create 4 dataset splits. What we call `train_ft_df` (Train Fine-Tune Set), `val_ft_df` (Validation Fine_tune Set), and `test_df` (Test Set) will contain labeled data. At the moment the examples are not labeled, but we will label them in Label Studio. The `train_df` is the unlabeled dataset we will be using Snorkel and a BERT model to label.

*   `fine_tune_df`: Labeled data we use for fine-tuning the bert model before putting model into a Snorkel Labeling Function.
*   `val_df`: This is our small dataset split we will use to measure the performance of the bert model trained with `fine_tune_df`.
*    `train_df`: As mentioned, the `train_df` is the unlabeled dataset we will be using Snorkel to label.
*    `test_df`: This is our dataset split for testing Snorkel's Label Model to get an estimate on how many labels were correctly labeled in our `train_df`.

In [ ]:
fine_tune_df, val_ft_df = train_test_split(labeled_df, 
                                           test_size=100, 
                                           random_state=42, 
                                           shuffle=True) # for fine-tuning BERT on subset of hand-labeled dataset
fine_tune_df, test_df = train_test_split(fine_tune_df, 
                                         test_size=500, 
                                         random_state=42, 
                                         shuffle=True) # for testing final model

In [5]:
# Encoding our labels for the labeling functions
ABSTAIN = -1
PHYSICAL_AND_METEOROLOGICAL_ENVIRONMENT = 0
SOIL_AND_SOIL_PRODUCTIVITY = 1
VEGETATION = 2
WATER_QUALITY_AND_QUANTITY = 3
FISH_AND_FISH_HABITAT = 4
WETLANDS = 5
WILDLIFE_AND_WILDLIFE_HABITAT = 6
SPECIES_AT_RISK = 7
GHG_EMISSIONS_AND_CLIMATE_CHANGE = 8
ASSESSMENT_OF_UPSTREAM_GHG_EMISSIONS = 9
AIR_EMISSIONS = 10
ACOUSTIC_ENVIRONMENT = 11
ELECTROMAGNETISM_AND_CORONA_DISCHARGE = 12
HUMAN_OCCUPANCY_AND_RESOURCE_USE = 13
HERITAGE_RESOURCES = 14
NAVIGATION_AND_NAVIGATION_SAFETY = 15
TRADITIONAL_LAND_AND_RESOURCE_USE = 16
SOCIAL_AND_CULTURAL_WELL_BEING = 17
HUMAN_HEALTH_AND_AESTHETICS = 18
INFRASTRUCTURE_AND_SERVICES = 19
EMPLOYMENT_AND_ECONOMY = 20
ENVIRONMENTAL_OBLIGATIONS = 21
RIGHTS_OF_INDIGENOUS_PEOPLES = 22

labels_list = [['Physical and meteorological environment',
                'Soil and soil productivity',
                'Vegetation',
                'Water quality and quantity',
                'Fish and fish habitat',
                'Wetlands',
                'Wildlife and wildlife habitat',
                'Species at Risk or Species of Special Status and related habitat',
                'Greenhouse gas (GHG) emissions and climate change',
                'GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions',
                'Air emissions',
                'Acoustic environment',
                'Electromagnetism and Corona Discharge',
                'Human occupancy and resource use',
                'Heritage resources',
                'Navigation and navigation safety',
                'Traditional land and resource use',
                'Social and cultural well-being',
                'Human health and aesthetics',
                'Infrastructure and services',
                'Employment and economy',
                'Environmental Obligations',
                'Rights of Indigenous Peoples']]

In [6]:
labels_list

[['Physical and meteorological environment',
  'Soil and soil productivity',
  'Vegetation',
  'Water quality and quantity',
  'Fish and fish habitat',
  'Wetlands',
  'Wildlife and wildlife habitat',
  'Species at Risk or Species of Special Status and related habitat',
  'Greenhouse gas (GHG) emissions and climate change',
  'GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions',
  'Air emissions',
  'Acoustic environment',
  'Electromagnetism and Corona Discharge',
  'Human occupancy and resource use',
  'Heritage resources',
  'Navigation and navigation safety',
  'Traditional land and resource use',
  'Social and cultural well-being',
  'Human health and aesthetics',
  'Infrastructure and services',
  'Employment and economy',
  'Environmental Obligations',
  'Rights of Indigenous Peoples']]

In [7]:
Physical_and_Meteorological_Environment = """Physical and Meteorological Environment
Precipitation
Snowfall
Wind
Mean temperatures
Slope
Geotechnical
Slumping
Subsidence
Weather
Erosion
Ice
Permafrost
Climate trend
ground
water erosion
wind erosion
acid-generating rock
temperature
physical
meteorological
landslides
mudflows
slumping
subsidence
seismicity
flooding
migrating watercourses
eroding banks
extreme weather events
peak flow regime
ice jams
acid rock
climate variability
ground conditions
thaw
"""


Soil_and_Soil_Productivity = """Soil Productivity
Soil
Agriculture
Topsoil
Subsoil
Soil horizon
Drainage
Erosion
Contamination
CCME
Canadian Council of Ministers of the Environment
Soil compaction
Soil structure
Soil classification
Soil handling
Containment
reclamation
thickness of horizon
tilth
grubbing
soil quality
salinity
sediments
rocks
minerals
sand"""

Vegetation = """Vegetation
Plant
planting
Rare plant
Boreal
Grassland
Prairie
Forest
forested
Clearing
Diversity
Abundance
Ecological
Inventory
Community
communities
ecological community
plant community
Cover
Weeds
Invasive species
invasive plants
Seed mix
Herbicide
Tree
Growth
Old growth
Biodiversity
forestry
clubroot
wood
spruce
fir
birch
pine
aspen
tamarack
willow
beech
maple
black walnut
hickory
oak
redcedar
hemlock
Douglas-fir
genus
agricultural"""


Water_Quality_and_Quantity = """Water Quality and Quantity
Water
Surface water
Ground water
Runoff
Contamination
contaminant
Water use
Hydrologic
hydrological
Withdrawal
Discharge
Flow
Peak
Basin
Inter-basin transfer
Water Quality
Water quality testing
Water table
Containment
Sediment
sewer
waste
groundwater
groundwater-related
aquifers
streamflow
acquatics
river
hydrometric""" 

Fish_and_Fish_Habitat = """Fish and Fish Habitat
Fish
Fisheries and Oceans Canada
DFO
Fisheries and Oceans
Local fisheries
Trout
Authorization
Fisheries Act
Offsetting
Monitoring
Instream work
in-stream work
Timing window
Restricted activity period
Fish-bearing water body
Riparian
Aquatic
Aquatic invasive species
Spawning
Migrating
Eggs
Fry
Sport fishery
Spawning deterrent
Stream
River
In-stream
instream
Wetted width
substrate
whale
marine mammal
salmon
oncorhynchus
walleye
pike
crappie
redhorse
pumpkinseed fish
bowfin
bass
catfish
sunfish
bluegill
spotted gar
muskellunge
ruffe
yellow perch
shiner
sucker
whitefish
cisco"""

Wetlands = """Wetlands
Class (wetland class)
wetland
Bog
Fen
Marsh
Swamp
Shallow water
Intermittent
Permanent
Non-permanent
Wetland function
Offsetting
Hydrological function
Drainage area
Canadian wetland classification system
Federal policy on wetland conservation
Conservation
Wetland monitoring
Reclamation
Compensation
water recharge
potholes
ponds
peatbogs
mires""" 

Wildlife_and_Wildlife_Habitat = """Wildlife and Wildlife Habitat
nocturnal
bat trees
hibernation
migratory birds
MBCA
migratory birds convention act
Nesting
Restricted activity period
Timing window
Mammal
Ungulate
Amphibian
Reptile
Breeding
Den
denning
Wintering
overwintering
Migration
Hibernaculum
hibernacula
Riparian habitat
Old growth habitat
Sensitive period
Sensory disturbance
Mortality
Mortality risk
Habitat alteration
Habitat loss
Habitat destruction
Range
Population
Distribution
Sanctuary
sanctuaries
Important Bird Area
Bat
caribou
bird
goose
amphibian
swans
frogs
toads
salamanders
newts
caecilians
furbearing
reptile
invertebrate
rattlesnake
snake
waterbird
bear
moose
bat
owl"""

Species_at_Risk = """Species at Risk
Species of Special Status
Rare
Sensitive
SARA
Species At Risk Act
COSEWIC
Committee on the Status of Endangered Wildlife in Canada
ECCC
Environment and Climate Change Canada
Critical habitat
Designation
Threshold
Schedule 1
At Risk
Endangered
Critical timing window
Restricted activity period
DFO
Fisheries and Oceans Canada
Canadian Wildlife Service
CWS
Recovery Strategy
Action Plan
Status
Listed
Permit
Offset
offsetting
Compensation
Caribou
Bat
special conservation status"""

Air_Emissions = """Air Emissions
CAC
criteria air contaminant
Emissions
Ground-level
Receptor
Model
modelled
modelling
Construction equipment
Vehicle
vehicular emissions
CCME
Volatile organic compounds
Combustion
Leak
Fugitive emissions
Detection
Flaring
Incinerating
incineration
Smoke
Venting
Pollute
pollutant
National Pollutant Release Inventory
Exceedance
Permit
Release
Ambient
Air quality
Hydrogen sulphide
H2S
particulate
so2"""

GHG_Emissions_and_Climate_Change = """GHG Emissions and Climate Change
greenhouse
greenhouse gas
green house gas
climate change
point source
area source
venting
fugitive
release
leak
burning
assumption
offset
carbon dioxide
CO2
CO2 equivalent
Target
Reduction
Percentage
Hinders
Net zero
net-zero
Reduce
Combustion emissions
International Standards Organization
ISO
methane
ghg
ozone
global warming"""

Assessment_of_Upstream_GHG_Emissions = """Assessment of Upstream GHG Emissions
upstream
quantitative
throughput
net zero
net-zero
Environment and Climate Change Canada (ECCC)
Venting
Threshold
CO2
Carbon dioxide
CO2 equivalent
Methane
emissions
steam
hydrogen
combustion
fugitive
venting
flaring"""

Acoustic_Environment = """Acoustic Environment
Sound
Noise
Receptor
Equipment
Frequency
Inaudible
Audible
Decibel
Notification
Noise control
Noise management
Model
modelling
Monitoring
monitor
db
acoustic"""


Environmental_Obligations = """Environmental Obligations
MBCA
migratory birds convention act
SARA
Species at risk act
DFO
Fisheries and Oceans Canada
Federal Wetland Policy
Hinder
Federal
Provincial
Territorial
International
Policy
Plan
Framework
Wildlife
Vegetation
Fish
Species at risk
Plant
law
legislation
regulations"""

Traditional_Land_and_Resource_Use = """Traditional Land and Resource Use
TLRU
Hunt
hunting
fishing
Harvest
harvesting
Culturally significant
Gather
Berries
Berry picking
Indigenous
Traditional ecological knowledge
indigenous knowledge
Elder
Trapping
trap
Ceremony
ceremonies
Medicinal
Cultural
Old growth
First Peoples
settlements
traditional territories
reserves
aboriginal
Knowledge Keeper
Trapping
trap
Spirit Bear
Spirit Animal
sacred area
sacred sites
metis
Métis
first nations
shxw’ōwhámel
lheidlit’enneh
whispering pines first nation
inuit
elders
kumik elder lodge
tribal
Abenaki
Innu
Montagnais-Naskapi
Oneida
Ahousaht
Interior Salish
Onondaga
Algonquin
Inuinnait
Copper Inuit
Pacheenaht
Assiniboine
Inuvialuit
Mackenzie Inuit
Petun
Atikamekw
Kainai
Piikani
Peigan
Baffin Island Inuit
K'asho Got'ine
Saldermiut Inuit
Beothuk
Kaska Dena
Sahtu Got'ine
Bearlake
Blackfoot Confederacy
Blackfoot
Kivallirmiut
Caribou Inuit
Secwepemc
Shuswap
Cayuga
Ktunaxa
Kootenay
Sekani
Central Coast Salish
Kwakwaka'wakw
Kwakiutl
Seneca
Coast Salish
Kyuquot and Checleseht
Shuta Got'ine
Cree
Labradormiut
Labrador Inuit
Siksika
Dakota
Lilwat
Lillooet
Slavey
Dakelh
Lingit
Tlingit
Stoney-Nakoda
Dane-zaa
Beaver
Syilx
Okanagan
Dene
Mi'kmaq
Tagish
Denesuline
Chipewyan
Mohawk
Tahltan
Ditidaht
Mowachaht-Muchalaht
Tla-o-qui-aht
Clayoquot
Ehattesaht
Nahani
Tlicho
Dogrib
Gitxsan
Gitksan
Netsilingmiut
Netsilik Inuit
Toquaht
Gwich'in
Neutral Confederacy
Tr'ondëk Hwëch'in (Han)
Haida
Nicola-Similkameen
Tseshaht
Sheshaht
Haisla
Kitamaat
Nisga'a
Tsilhqot'in
Chilcotin
Haudenosaunee
Six Nations
Iroquois
Nlaka'pamux
Thompson
Tsimshian
Heiltsuk
Northern Georgia Strait Coast Salish
Tsuut'ina
Sarcee
Hesquiaht
Nuchatlaht
Tutchone
Hupacasath
Opetchesaht
Nunavimmiut
Ungava Inuit
Uchucklesaht
Huu-ay-aht
Nuu-chah-nulth
Ucluelet
Huron-Wendat
Nuxalk
Bella Coola
Wolastoqiyik
Maliseet
Iglulingmuit
Iglulik Inuit
Odawa
Wetal
Tsetsaut
Inuit
Ojibwa
Yellowknives
popkum first nation
leq’á:mel first nation
alexander first nation
samson cree first nation
o’chiese first nation
ermineskin cree nation
enoch cree nation
indian
eskimo"""

Electromagnetism_and_Corona_Discharge = """Electromagnetism and Corona Discharge
voltage
ozone concentration
eletric
corona
discharge
magnetic
power line
electromagnetic
signals
maximum load
induction
frequency
inteference
radio
television
foul weather
ambient conditions
240 kV"""

Human_Occupancy_and_Resource_Use = """Human Occupancy and Resource Use
residents"""

Heritage_Resources = """Heritage Resources"""

Navigation_and_Navigation_Safety = """Navigation and Navigation Safety"""

Social_and_Cultural_Well_Being = """Social and Cultural Well-Being"""

Human_Health_and_Aesthetics = """Human Health and Aesthetics"""

Infrastructure_and_Services = """Infrastructure and Services"""

Employment_and_Economy = """Employment and Economy"""

Rights_of_Indigenous_Peoples = """Rights of Indigenous Peoples
Traditional Knowledge
Indigenous Knowledge
Aboriginal Knowledge
Aboriginal
Hunt
hunting
fishing
Harvest
harvesting
Culturally significant
Culturally modified tree
Gather
Berries
Medicine
Berry picking
Indigenous
Elder
Trapping
engagement
trap
Ceremony
ceremonies
Medicinal
Cultural
First Peoples
indian act
treaty
rights-bearing
indigenous rights
reserves
aboriginal
indigenous
metis
Métis
first nations
shxw’ōwhámel
lheidlit’enneh
whispering pines first nation
inuit
elders
kumik elder lodge
tribal
Abenaki
Innu
Montagnais-Naskapi
Oneida
Ahousaht
Interior Salish
Onondaga
Algonquin
Inuinnait
Copper Inuit
Pacheenaht
Assiniboine
Inuvialuit
Mackenzie Inuit
Petun
Atikamekw
Kainai
Piikani
Peigan
Baffin Island Inuit
K'asho Got'ine
Saldermiut Inuit
Beothuk
Kaska Dena
Sahtu Got'ine
Bearlake
Blackfoot Confederacy
Blackfoot
Kivallirmiut
Caribou Inuit
Secwepemc
Shuswap
Cayuga
Ktunaxa
Kootenay
Sekani
Central Coast Salish
Kwakwaka'wakw
Kwakiutl
Seneca
Coast Salish
Kyuquot and Checleseht
Shuta Got'ine
Cree
Labradormiut
Labrador Inuit
Siksika
Dakota
Lilwat
Lillooet
Slavey
Dakelh
Lingit
Tlingit
Stoney-Nakoda
Dane-zaa
Beaver
Syilx
Okanagan
Dene
Mi'kmaq
Tagish
Denesuline
Chipewyan
Mohawk
Tahltan
Ditidaht
Mowachaht-Muchalaht
Tla-o-qui-aht
Clayoquot
Ehattesaht
Nahani
Tlicho
Dogrib
Gitxsan
Gitksan
Netsilingmiut
Netsilik Inuit
Toquaht
Gwich'in
Neutral Confederacy
Tr'ondëk Hwëch'in (Han)
Haida
Nicola-Similkameen
Tseshaht
Sheshaht
Haisla
Kitamaat
Nisga'a
Tsilhqot'in
Chilcotin
Haudenosaunee
Six Nations
Iroquois
Nlaka'pamux
Thompson
Tsimshian
Heiltsuk
Northern Georgia Strait Coast Salish
Tsuut'ina
Sarcee
Hesquiaht
Nuchatlaht
Tutchone
Hupacasath
Opetchesaht
Nunavimmiut
Ungava Inuit
Uchucklesaht
Huu-ay-aht
Nuu-chah-nulth
Ucluelet
Huron-Wendat
Nuxalk
Bella Coola
Wolastoqiyik
Maliseet
Iglulingmuit
Iglulik Inuit
Odawa
Wetal
Tsetsaut
Inuit
Ojibwa
Yellowknives
popkum first nation
leq’á:mel first nation
alexander first nation
samson cree first nation
o’chiese first nation
ermineskin cree nation
enoch cree nation
indian
eskimo
"""

SyntaxError: invalid syntax (<ipython-input-7-2871ccb4a7d9>, line 353)

In [9]:
keywords = [Physical_and_Meteorological_Environment,
           Soil_and_Soil_Productivity,
           Vegetation,
           Water_Quality_and_Quantity,
           Fish_and_Fish_Habitat,
           Wetlands,
           Wildlife_and_Wildlife_Habitat,
           Species_at_Risk,
           Air_Emissions,
           GHG_Emissions_and_Climate_Change,
           Assessment_of_Upstream_GHG_Emissions,
           Acoustic_Environment,
           Environmental_Obligations,
           Traditional_Land_and_Resource_Use]

keywords = [x.lower().split('\n') for x in keywords]

In [10]:
keywords[1]

['soil',
 'agriculture',
 'topsoil',
 'subsoil',
 'soil horizon',
 'drainage',
 'erosion',
 'contamination',
 'ccme',
 'canadian council of ministers of the environment',
 'soil compaction',
 'soil structure',
 'soil classification ',
 'soil handling',
 'containment',
 'reclamation']

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

stemmer = PorterStemmer()

for i, label_keywords in enumerate(keywords):
    keywords[i] = [w for w in label_keywords if w not in stopwords.words("english")]
    keywords[i] = [stemmer.stem(w) for w in keywords[i]]

keywords

[['physical and meteorological environ',
  'precipit',
  'snowfal',
  'wind',
  'mean temperatur',
  'slope',
  'geotechn',
  'slump',
  'subsid',
  'weather',
  'eros',
  'ice',
  'permafrost',
  'climate trend'],
 ['soil',
  'agricultur',
  'topsoil',
  'subsoil',
  'soil horizon',
  'drainag',
  'eros',
  'contamin',
  'ccme ',
  'canadian council of ministers of the environ',
  'soil compact',
  'soil structur',
  'soil classification ',
  'soil handl',
  'contain',
  'reclam'],
 ['veget',
  'plant',
  'plant',
  'rare pl',
  'boreal',
  'grassland',
  'prairie ',
  'forest',
  'forest',
  'clear',
  'divers',
  'abund',
  'ecological ',
  'inventory ',
  'classif',
  'commun',
  'commun',
  'ecological commun',
  'plant commun',
  'cover',
  'weed',
  'invasive speci',
  'invasive plants ',
  'seed mix',
  'herbicid',
  'tree',
  'growth ',
  'old growth',
  'biodivers'],
 ['water',
  'surface wat',
  'ground wat',
  'runoff',
  'contamin',
  'contamin',
  'water us',
  'hydrolog'

## Loading the Labeled Data from the Zero-Shot Classification

In [1]:
import pickle
pickled_dataset = 'zero_shot_vcs_1k_for_labeling'
infile = open(f'./data/{pickled_dataset}.pkl', 'rb')
zs_vcs_dict_list = pickle.load(infile)
infile.close()

In [2]:
zs_vcs_dict_list[0:2]

[{'sequence': 'TABLE 8A2 REASONABLY FORSEEABLE DEVELOPMENTS IDENTIFIED WITHIN THE SPATIAL BOUNDARIES OF THE BEAR CANYON SECTION Map ID1 Title Proponent Description Construction Schedule2 Assessment Type Quantitative or Qualitative Physical Environment and Soils LSA Water Quality and Quantity RSA Air Quality RSA Acoustic Environment RSA Fish and Fish Habitat RSA Wetlands RSA Vegetation RSA Wildlife and Wildlife Habitat RSA HORU RSA Heritage Resources RSA TRLU RSA Community RSA 13307912 W6M to 6367913 W6M Trioil Resources Ltd Pipeline Quantitative 13307912 W6M to 13307912 W6M Trioil Resources Ltd Pipeline Quantitative 634876 W6M to 1034876 W6M Venturion Oil Limited Pipeline Quantitative 1234876 W6M to 634876 W6M Venturion Oil Limited Pipeline Quantitative 634876 W6M to 1234876 W6M Venturion Oil Limited Pipeline Quantitative 1234876 W6M to 634876 W6M Venturion Oil Limited Pipeline Quantitative 1034876 W6M',
  'labels': ['Wildlife and wildlife habitat',
   'Water quality and quantity',
   

In [26]:
filtered_zs_vcs = []

for zs_vcs_i in zs_vcs_dict_list[:10]:
    num_of_labels = len([filtered_score for filtered_score in zs_vcs_i['scores'] if filtered_score >= 0.8])
    filtered_zs_vcs.append(zs_vcs_i['sequence'], 
                           zs_vcs_i['labels'][:num_of_labels], 
                           zs_vcs_i['scores'][:num_of_labels])

filtered_zs_vcs

TypeError: append() takes exactly one argument (3 given)

In [24]:
[filtered_score for filtered_score in zs_vcs_dict_list[0]['scores'] if filtered_score >= 0.8]

[0.9908032417297363,
 0.9892772436141968,
 0.9859840869903564,
 0.9473797082901001,
 0.9381691217422485,
 0.8850263953208923,
 0.8406775593757629]

In [25]:
zs_vcs_dict_list[0]['scores'][:7]

[0.9908032417297363,
 0.9892772436141968,
 0.9859840869903564,
 0.9473797082901001,
 0.9381691217422485,
 0.8850263953208923,
 0.8406775593757629]

## Creating the Snorkel Functions

In [ ]:
from snorkel.labeling import labeling_function

@labeling_function
def keyword_lookup(x, keywords, label):
    if any(word in x.text for word in keywords):
        return label
    return ABSTAIN



In [8]:
from snorkel.labeling.model import MajorityLabelVoter
from sklearn.preprocessing import MultiLabelBinarizer

Y = labels_list

# fit a MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit_transform(Y)

# create a majority vote model and predict
majority_model = MajorityLabelVoter(cardinality=23)

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1]])

In [ ]:
from snorkel.labeling import MajorityLabelVoter
from sklearn.preprocessing import MultiLabelBinarizer

Y = labels_list

# fit a MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit_transform(Y)

# create a majority vote model and predict
majority_model = MajorityLabelVoter(cardinality=23)
predictions = majority_model.predict_proba(L=L_test)
df_multilabel = pd.DataFrame()
df_multilabel['predict_proba'] = predictions.tolist()

# get all the non zero indices which are the multi labels
df_multilabel['multi_labels'] = df_multilabel['predict_proba'].apply(lambda x: np.nonzero(x)[0])
    
#transform to mlb for classification report
df_multilabel['mlb_pred'] = df_multilabel['multi_labels'].apply(lambda x: mlb.transform([x])[0])
print(df_multilabel.head())

#convert to str in order to see how many multi labels did we gain
multi_label_string = df_multilabel.multi_labels.apply(lambda x: ", ".join(le.inverse_transform(x)))
print(multi_label_string.value_counts()[:50])

# print some metrics using classification report 
y_pred = df_multilabel.mlb_pred.apply(lambda x: list(x)).to_numpy().tolist()
y_true = mlb.transform(Y.values).tolist()
print(classification_report(y_true, y_pred, target_names = mlb.classes_))